# K-Nearest Neighbors Classification

The dataset was obtained from https://archive.ics.uci.edu/ml/datasets/Iris.

In [1]:
import numpy as np
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from collections import Counter

In [2]:
# read data and parse it
data = pd.read_csv('../datasets/movie_reviews.csv')
data = data[['fold_id', 'text', 'tag']]

In [3]:
# Splitting data 70 % train, 30 % test
train_df = data.loc[data['fold_id'] < 7]
test_df = data.loc[data['fold_id'] >= 7]

In [4]:
def calculate_accuracy(y_hat, y):
    return np.sum(np.equal(y, y_hat), dtype=float) / len(y)

In [5]:
class NaiveBayesText:
    
    def __init__(self, train_df):
        """
        train_df (pandas.DataFrame) : Training dataset
        train_df must have a column called 'tag' representing the output class,
            and a column called 'text'
        """
        self.train_df = train_df
        self.clean_documents()
        self.tags = list(set(train_df.tag.values))
        self.tag_vocabulary_counts = {tag : Counter(self.get_vocabulary(
            self.train_df.loc[self.train_df['tag'] == tag])) for tag in self.tags}
        self.total_vocabulary_size = len(self.get_vocabulary(self.train_df))
        
    def clean_documents(self):
        waste = '\d|\"|\'|!|\)|\(|\.|,|' + '\\b(' + \
            '|'.join(stopwords.words('english')) + ')\\W'
        self.train_df.loc[:, self.train_df.columns == 'text'] = self.train_df.text.apply(
            lambda doc : self.clean_document(doc, waste))

    def clean_document(self, doc, waste):
        """ Removes useless filler words/punctuations from text """
        return re.sub(waste, '', doc).split()
    
    def get_vocabulary(self, df):
        vocabulary = []
        for doc_list in df.text.values:
            for word in doc_list:
                vocabulary.append(word)
        return vocabulary
    
    def prob_tag(self, tag):
        return len(self.train_df.loc[self.train_df['tag'] == tag])/len(self.train_df)

    def prob_doc_given_tag(self, doc, tag):
        prob_words = np.array([self.prob_word_given_tag(word, tag) 
                               for word in doc.split()])
        return np.prod(prob_words) * self.prob_tag(tag)

    def prob_word_given_tag(self, word, tag):
        word_count = self.tag_vocabulary_counts[tag][word]
        return (word_count + 1)/(len(self.tag_vocabulary_counts) + self.total_vocabulary_size)

    def predict_tag(self, doc):
        prob_tags = np.array([self.prob_doc_given_tag(doc, tag) for tag in self.tags])
        return self.tags[np.argmax(prob_tags)]
    
    def predict_batch(self, docs):
        return np.array([self.predict_tag(doc) for doc in docs])

In [6]:
# Use the best parameter
nbt = NaiveBayesText(train_df)
y_hat = nbt.predict_batch(test_df['text'])
test_acc = calculate_accuracy(test_df['tag'], y_hat)
print("Test set accuracy: {:1.3f}%".format(test_acc * 100))

/home/pranaymethuku/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


ValueError: the 'dtype' parameter is not supported in the pandas implementation of sum()